In [1]:
#on récupère la liste de toutes les stations
import fonctions as f
client=f.Client()

station=client.request('GET', 'https://public-api.meteofrance.fr/public/DPObs/v1/liste-stations', verify=False)



dataStation = f.StringIO(station.text)
print(dataStation)
df = f.pd.read_csv(dataStation,sep=';')
df = df.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'})



In [5]:
gdf = f.gpd.GeoDataFrame(
    df, geometry=f.gpd.points_from_xy(df.lon, df.lat), crs=4326
)
gdf

,Id_station,Id_omm,Nom_usuel,lat,lon,Altitude,Date_ouverture,Pack,geometry
0,1014002,NaN,ARBENT,46.278167,5.669000,534,2003-10-01,RADOME,POINT (5.669 46.27817)
1,1027003,NaN,BALAN_AERO,45.833000,5.106667,196,2014-05-26,ETENDU,POINT (5.10667 45.833)
2,1034004,NaN,BELLEY,45.769333,5.688000,330,2001-09-13,RADOME,POINT (5.688 45.76933)
3,1064001,NaN,VERIZIEU,45.777167,5.487167,281,2015-01-01,ETENDU,POINT (5.48717 45.77717)
4,1071001,NaN,CESSY,46.310333,6.080333,507,2002-05-01,RADOME,POINT (6.08033 46.31033)
...,...,...,...,...,...,...,...,...,...
2118,98832004,91588.0,MTGNE SOURCES,-22.143833,166.593167,773,1989-08-01,ETENDU,POINT (166.59317 -22.14383)
2119,98832005,NaN,OUINNE,-21.984000,166.680500,54,1974-01-01,ETENDU,POINT (166.6805 -21.984)
2120,98832006,NaN,RIVIERE BLANCHE,-22.132667,166.726333,171,2000-11-01,ETENDU,POINT (166.72633 -22.13267)
2121,98832101,NaN,GORO_ANCIENNE_PEPINIERE,-22.269167,166.967500,298,1995-01-01,ETENDU,POINT (166.9675 -22.26917)


In [6]:
#On récupere la liste des stations climatologiques avec les mesures à une fréquence de 1 journée
client = f.Client()
 # Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': 'application/json'})
response = client.request('GET', 'https://public-api.meteofrance.fr/public/DPClim/v1/liste-stations/infrahoraire-6m?id-departement=91', verify=False)
df=f.pd.DataFrame(response.json())
print(df.head())








         id            nom  posteOuvert  typePoste       lon        lat  alt  \
0  91027002           ORLY         True          0  2.397000  48.718000   86   
1  91069001    BOIGNEVILLE        False          2  2.381167  48.325500  115   
2  91103001  BRETIGNY_SAPC         True          2  2.326000  48.600500   78   
3  91184001   COURDIMANCHE         True          1  2.374333  48.421833   71   
4  91200002        DOURDAN         True          1  1.993000  48.526167  116   

   postePublic  
0         True  
1         True  
2         True  
3         True  
4         True  


In [7]:
gdf = f.gpd.GeoDataFrame(
    df, geometry=f.gpd.points_from_xy(df.lon, df.lat), crs=4326
)
gdf



,id,nom,posteOuvert,typePoste,lon,lat,alt,postePublic,geometry
0,91027002,ORLY,True,0,2.397000,48.718000,86,True,POINT (2.397 48.718)
1,91069001,BOIGNEVILLE,False,2,2.381167,48.325500,115,True,POINT (2.38117 48.3255)
2,91103001,BRETIGNY_SAPC,True,2,2.326000,48.600500,78,True,POINT (2.326 48.6005)
3,91184001,COURDIMANCHE,True,1,2.374333,48.421833,71,True,POINT (2.37433 48.42183)
4,91200002,DOURDAN,True,1,1.993000,48.526167,116,True,POINT (1.993 48.52617)
5,91275001,GOMETZ-LE-CHAT.,True,2,2.135833,48.676833,159,True,POINT (2.13583 48.67683)
6,91405002,MILLY-LA-FORET,True,2,2.431833,48.382167,109,True,POINT (2.43183 48.38217)
7,91679001,VILLIERS-LE-BAC,False,2,2.111500,48.730333,147,True,POINT (2.1115 48.73033)
8,91679002,VILLIERS-LE-BACLE2,False,2,2.116167,48.735333,161,True,POINT (2.11617 48.73533)


In [8]:
#Représentation des stations météos
m=f.interactive_map_dpe(gdf)
m


In [9]:
#générer la base de données selon certaines dates

client=f.Client()
datameteo=client.request('GET',"https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde={}".format(f.gener_idcommande("2024-11-01","2024-11-05")),verify=False)
print(datameteo)


<Response [201]>


In [10]:

def gener_data(date_deb, date_fin):
    client=f.Client()
    datameteo=client.request('GET',"https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde={}".format(f.gener_idcommande(date_deb,date_fin)),verify=False)
    print(datameteo)
    data = f.StringIO(datameteo.text)
    print(data)
    f.time.sleep(5)
    return f.pd.read_csv(data,sep=';')

In [11]:
matrix2022=f.generate_month_matrix(2022)
matrix2023=f.generate_month_matrix(2023)
matrix2024=f.generate_month_matrix(2024)

matrix2024=matrix2024[:10]

In [12]:
matrix2024

[(datetime.date(2024, 1, 5), datetime.date(2024, 1, 31)),
 (datetime.date(2024, 2, 1), datetime.date(2024, 2, 28)),
 (datetime.date(2024, 3, 1), datetime.date(2024, 3, 31)),
 (datetime.date(2024, 4, 1), datetime.date(2024, 4, 30)),
 (datetime.date(2024, 5, 1), datetime.date(2024, 5, 31)),
 (datetime.date(2024, 6, 1), datetime.date(2024, 6, 30)),
 (datetime.date(2024, 7, 1), datetime.date(2024, 7, 31)),
 (datetime.date(2024, 8, 1), datetime.date(2024, 8, 31)),
 (datetime.date(2024, 9, 1), datetime.date(2024, 9, 30)),
 (datetime.date(2024, 10, 1), datetime.date(2024, 10, 31))]

In [16]:
data2022=f.pd.DataFrame()
for i in range(len(matrix2022)):
    data2022=f.pd.concat([data2022,gener_data(matrix2022[i][0],matrix2022[i][1])], ignore_index=True)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [17]:
data2023=f.pd.DataFrame()
for i in range(len(matrix2023)):
    data2023=f.pd.concat([data2023,gener_data(matrix2023[i][0],matrix2023[i][1])], ignore_index=True)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [14]:
data2024=f.pd.DataFrame()
for i in range(len(matrix2024)):
    data2024=f.pd.concat([data2024,gener_data(matrix2024[i][0],matrix2024[i][1])], ignore_index=True)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [21]:
datameteo=f.pd.concat([data2022,data2023], ignore_index=True)
datameteo=f.pd.concat([datameteo,data2024], ignore_index=True)


In [19]:
# On convertit la base de données existante en fichier cvs 
datameteo.to_csv('data.csv', index=False)

Général
POSTE : Code ou identifiant de la station météorologique (ou du poste de mesure).
DATE : Date et heure des relevés météorologiques.

Précipitations et Humidité
RR : Cumul des précipitations en millimètres (pluie tombée sur une période donnée).
DRR : Durée des précipitations (en heures ou minutes).
UM : Humidité moyenne (en %).
HUN : Heure de l'humidité minimale.
UX : Humidité maximale observée (%).
HUX : Heure de l'humidité maximale.
DHUMI40/DHUMI80 : Durée (en heures) pendant laquelle l’humidité relative dépasse 40% ou 80%.
DHUMEC : Durée d’humidité critique (périodes favorables à certaines conditions climatiques spécifiques, ex. moisissures).


Température
TN : Température minimale (°C).
HTN : Heure à laquelle la température minimale a été observée.
TX : Température maximale (°C).
HTX : Heure à laquelle la température maximale a été observée.
TM : Température moyenne (°C).
TMNX : Moyenne des températures minimales et maximales.
TNSOL : Température minimale au sol (°C).
TN50 : Température minimale à 50 cm du sol (°C).
TAMPLI : Amplitude thermique (écart entre température maximale et minimale).
TNTXM : Moyenne de la température minimale et maximale sur une période donnée.
TSVM : Température moyenne au sol.


Pression atmosphérique
PMERM : Pression moyenne au niveau de la mer (hPa).
PMERMIN : Pression minimale enregistrée au niveau de la mer.


Vents
FFM : Vitesse moyenne du vent (m/s ou km/h).
FXI : Rafale maximale de vent (intensité maximale, en m/s ou km/h).
DXI : Direction de la rafale maximale (en degrés).
HXI : Heure de la rafale maximale.
FF2M : Vitesse moyenne du vent à 2 mètres du sol.
FXI2/DXI2/HXI2 : Rafale maximale, direction et heure correspondantes à 2 mètres du sol.
FXI3S/DXI3S/HXI3S : Rafale maximale sur 3 secondes, direction et heure correspondantes.


Conditions météorologiques
NEIG : Indicateur de chute de neige (binaire ou cumul).
BROU : Indicateur de brouillard (présence ou non).
ORAG : Indicateur d'orage (binaire ou intensité).
GRESIL/GRELE : Présence de grésil ou de grêle.
ROSEE : Présence de rosée.
VERGLAS : Indicateur de verglas.
SOLNEIGE : Hauteur de neige au sol (cm).
GELEE : Indicateur de gelée au sol.
FUMEE : Indicateur de fumée dans l'air.
BRUME : Indicateur de brume.
ECLAIR : Observation d’éclairs ou d'activité orageuse.


Évapotranspiration
ETPMON : Évapotranspiration mensuelle (mm).
ETPGRILLE : Évapotranspiration potentielle sur grille (modélisation).


Indices UV
UV_INDICEX : Indice UV maximal observé (valeur de 0 à 11+).
UV : Indice UV instantané.


Conditions maritimes
TMERMAX : Température maximale de la mer (°C).
TMERMIN : Température minimale de la mer (°C).


Neige
HNEIGEF : Hauteur de neige fraîche tombée (cm).
NEIGETOTX : Cumul maximal de neige (cm).
NEIGETOT06 : Cumul de neige sur 6 heures.


Divers
INST : Insolation (durée d’ensoleillement, en heures).
GLOT : Globale totale (rayonnement global reçu, en W/m²).
DIFT : Diffus totale (partie diffusée du rayonnement global, en W/m²).
DIRT : Direct totale (rayonnement direct reçu, en W/m²).
SIGMA : Écart-type ou variabilité des mesures.
INFRART : Rayonnement infrarouge terrestre.
NB300/BA300 : Nombre ou durée d'éclairements UV > 300 nm (en minutes ou heures).